# Density × speed-transfer decay sweep

Sweeps density (controls agents per group) and speed-transfer decay to see how much rhythmicity leaks into the nominally non-rhythmic group.

- Agents per group = `500 * density_factor` (so 0.40 ≈ 200 agents).
- Day duration: 200 steps; simulation length: 800 steps (4 days).
- Defaults otherwise unchanged; `.mat` saving disabled for speed.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from simulation import (
    AblationConfig,
    OutputOptions,
    SimulationParameters,
    default_simulation_config,
    run_ablation_study,
)

plt.style.use("seaborn-v0_8")

In [ ]:
# Sweep definition
density_factors = [0.20, 0.40, 0.80]
speed_transfer_decays = [0.0, 0.4, 0.8, 0.95]
rhythmicity_permutations = 100

# Base config: 4 simulated days, 200-step day length
cfg = default_simulation_config()
cfg.sim = SimulationParameters(day_duration=200, sim_duration=800)
cfg.output = OutputOptions(output_dir=Path("visualizations/output"))

In [ ]:
records = []
rng = np.random.default_rng(42)

for density in density_factors:
    for decay in speed_transfer_decays:
        label = f"d{density:.2f}_decay{decay:.2f}"
        ablations = {label: AblationConfig(speed_transfer_decay=decay)}
        results = run_ablation_study(
            density_factor=density,
            ablations=ablations,
            config=cfg,
            rhythmicity_permutations=rhythmicity_permutations,
            rng=rng,
        )
        summary = results[label]
        records.append(
            {
                "label": label,
                "density_factor": density,
                "agents_per_group": 500 * density,
                "speed_transfer_decay": decay,
                "amplitude_group1": summary.amplitude_group1,
                "amplitude_group2": summary.amplitude_group2,
                "phase_group1": summary.phase_group1,
                "phase_group2": summary.phase_group2,
                "phase_shift": summary.phase_shift_g2_minus_g1,
                "p_value_group1": summary.rhythmicity_p_value_group1,
                "p_value_group2": summary.rhythmicity_p_value_group2,
            }
        )

df = pd.DataFrame(records)
df.sort_values(["density_factor", "speed_transfer_decay"], inplace=True)
df

In [ ]:
# Heatmaps for non-rhythmic group metrics
pivot_amp = df.pivot(
    index="density_factor", columns="speed_transfer_decay", values="amplitude_group2"
).sort_index()
pivot_phase = df.pivot(
    index="density_factor", columns="speed_transfer_decay", values="phase_shift"
).sort_index()

fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)

im0 = axes[0].imshow(pivot_amp.values, origin="lower", aspect="auto")
axes[0].set_xticks(range(len(pivot_amp.columns)))
axes[0].set_xticklabels([f"{c:.2f}" for c in pivot_amp.columns])
axes[0].set_yticks(range(len(pivot_amp.index)))
axes[0].set_yticklabels([f"{r:.2f}" for r in pivot_amp.index])
axes[0].set_xlabel("speed_transfer_decay")
axes[0].set_ylabel("density_factor")
axes[0].set_title("Amplitude (Group 2)")
fig.colorbar(im0, ax=axes[0], label="speed amplitude")

im1 = axes[1].imshow(pivot_phase.values, origin="lower", aspect="auto", cmap="twilight")
axes[1].set_xticks(range(len(pivot_phase.columns)))
axes[1].set_xticklabels([f"{c:.2f}" for c in pivot_phase.columns])
axes[1].set_yticks(range(len(pivot_phase.index)))
axes[1].set_yticklabels([f"{r:.2f}" for r in pivot_phase.index])
axes[1].set_xlabel("speed_transfer_decay")
axes[1].set_title("Phase shift (Group2 - Group1, degrees)")
fig.colorbar(im1, ax=axes[1], label="degrees")

plt.show()

In [ ]:
# Line view: amplitude of non-rhythmic group across decay for each density
fig, ax = plt.subplots(figsize=(8, 4))
for density in density_factors:
    subset = df[df["density_factor"] == density]
    ax.plot(
        subset["speed_transfer_decay"],
        subset["amplitude_group2"],
        marker="o",
        label=f"density {density:.2f} (agents≈{int(500*density)})",
    )
ax.set_xlabel("speed_transfer_decay")
ax.set_ylabel("Amplitude (Group 2)")
ax.set_title("Non-rhythmic group entrainment vs decay")
ax.legend()
plt.tight_layout()
plt.show()